In [1]:
import pandas as pd
df = pd.read_csv('../data/us_etf_data/SPY.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.896694,480500
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.953669,201300
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,27.238594,529400
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,27.352570,531500


In [3]:
price_df = df.loc[:,['Date','Adj Close']].copy()
price_df.head()

,Date,Adj Close
0,1993-01-29,26.706757
1,1993-02-01,26.896694
2,1993-02-02,26.953669
3,1993-02-03,27.238594
4,1993-02-04,27.352570


In [4]:
price_df.set_index(['Date'],inplace=True)
price_df.head()

,Adj Close
Date,
1993-01-29,26.706757
1993-02-01,26.896694
1993-02-02,26.953669
1993-02-03,27.238594
1993-02-04,27.352570


In [5]:
price_df['center'] = price_df['Adj Close'].rolling(window = 20).mean()
price_df.iloc[18:25]

,Adj Close,center
Date,,
1993-02-25,26.953669,NaN
1993-02-26,26.991682,26.910942
1993-03-01,26.915695,26.921389
1993-03-02,27.314577,26.942283
1993-03-03,27.428545,26.966027
1993-03-04,27.276592,26.967927
1993-03-05,27.200619,26.960329


In [6]:
price_df['ub'] = price_df['center'] + 2 * price_df['Adj Close'].rolling(window = 20).std()
price_df.iloc[18:25]

,Adj Close,center,ub
Date,,,
1993-02-25,26.953669,NaN,NaN
1993-02-26,26.991682,26.910942,27.598068
1993-03-01,26.915695,26.921389,27.601764
1993-03-02,27.314577,26.942283,27.644772
1993-03-03,27.428545,26.966027,27.701465
1993-03-04,27.276592,26.967927,27.706519
1993-03-05,27.200619,26.960329,27.685261


In [7]:
price_df['lb'] = price_df['center'] - 2 * price_df['Adj Close'].rolling(window = 20).std()
price_df.iloc[18:25]

,Adj Close,center,ub,lb
Date,,,,
1993-02-25,26.953669,NaN,NaN,NaN
1993-02-26,26.991682,26.910942,27.598068,26.223816
1993-03-01,26.915695,26.921389,27.601764,26.241013
1993-03-02,27.314577,26.942283,27.644772,26.239794
1993-03-03,27.428545,26.966027,27.701465,26.230589
1993-03-04,27.276592,26.967927,27.706519,26.229334
1993-03-05,27.200619,26.960329,27.685261,26.235397


In [8]:
n = 20
sigma = 2
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = price_df['Adj Close'].rolling(n).mean() #중앙 이동 평균선
    bb['ub'] = bb['center'] + sigma * price_df['Adj Close'].rolling(n).std() # 상단 밴드
    bb['lb'] = bb['center'] - sigma * price_df['Adj Close'].rolling(n).std() # 하단 밴드
    return bb
bollinger = bollinger_band(price_df, n, sigma)

In [9]:
base_date = '2009-01-02'
sample = bollinger.loc[base_date:]
sample.head()

,Adj Close,center,ub,lb
Date,,,,
2009-01-02,75.099487,71.378963,74.538229,68.219698
2009-01-05,75.010582,71.711677,74.931608,68.491746
2009-01-06,75.511505,71.964058,75.543401,68.384716
2009-01-07,73.249435,71.980327,75.580937,68.379718
2009-01-08,73.548378,72.071645,75.736733,68.406557


In [10]:
sample = price_df.loc[base_date:]

In [12]:
book = sample[['Adj Close']].copy()
book['trade'] = '' #거래내역 칼럼
book.head()

,Adj Close,trade
Date,,
2009-01-02,75.099487,
2009-01-05,75.010582,
2009-01-06,75.511505,
2009-01-07,73.249435,
2009-01-08,73.548378,


In [13]:
def create_trade_book(sample):
    book = sample[['Adj Close']].copy()
    book['trade'] = ''
    return (book)

In [ ]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, 'Adj Close'] > sample.loc[i, 'ub']: # 상단 밴드 이탈 시 동작 안 함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb'] > sample.loc[i, 'Adj Close']: # 하반 밴드 이탈 시 매수 대기
            if book.shift(1).loc[i, 'trade'] = = 'buy': # 이미 매수상태라면
                book.loc[i, 'trade'] = 'buy' # 매수상태 유지
            else:
                book.loc[i, 'trade'] = 'ready' # 대기
        elif sample.loc[i, 'ub'] >= sample.loc[i, 'Adj Close'] and sample.loc[i, 'AdjClose'] >= sample.loc[i, 'lb']: # 볼린저 밴드 안에 있을 때
            if book.shift(1).loc[i, ‘trade’] = = ‘buy’ or book.shift(1).loc[i, ‘trade’] ==‘ready’:
                book.loc[i, 'trade'] = 'buy' # 매수상태 유지
            else:
                book.loc[i, 'trade'] = '' # 동작 안 함
    return (book)